# Import

In [1]:
%matplotlib widget

In [2]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Define

In [3]:
def my_plot(table, plot_args, title, xlabel='iteration', ylabel='accuracy', ylim=[0, 1]):
    fig, ax = plt.subplots(2, 1)
    
    lo = ax[0].plot(
        table.columns.values,
        table.transpose(),
        **plot_args
    )
    ax[0].legend(iter(lo), table.index.values, loc='best')

    ax[0].set_title(title)
    ax[0].set_xlabel(xlabel)
    ax[0].set_ylabel(ylabel)
    ax[0].set_ylim(*ylim)
    
    cell_text = []
    order = ['baseline', 'LotS', 'LitL']
    for treat in order:
        display_text = [f'{acc*100:.2f}%' for acc in table.loc[treat, :]]
        cell_text.append(display_text)
    
    ax[1].table(cellText=cell_text, colLabels=table.columns, rowLabels=order, loc='center')
    ax[1].axis('off')
    
    plt.tight_layout()
    return fig
    

In [4]:
def display_val_summary(modifier, iteration, plot_args, eval_dir, ylim=[0,1], save_plot=False, plot_type='jpg'):
    fname = os.path.join(eval_dir, f'r{iteration}', 'tables', f'configs.{modifier}.csv')
    summary_table = pd.read_csv(fname, index_col = 0)
    summary_table = summary_table[[str(n) for n in range(1, iteration+1)]]
    
    print(summary_table)
    fig = my_plot(summary_table, plot_args, modifier, ylim=ylim)
    
    if save_plot:
        assert not plot_type is None
        os.makedirs(os.path.join(eval_dir, f'r{iteration}', 'sum_plots'), exist_ok=True)
        fig.savefig(os.path.join(eval_dir, f'r{iteration}', 'sum_plots', f'configs.{modifier}.{plot_type}'))
        
        print(f"Saved at\n{os.path.join(eval_dir, f'r{iteration}', 'sum_plots')}")


def display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, combined, ylim=[0,1], save_plot=False, plot_type='jpg'):
    rep = {
        '/': '-',
        ';': '--',
    }
    
    fname_key = '.'.join(sub_keys.values())
    for old_char, new_char in rep.items():
        fname_key = fname_key.replace(old_char, new_char)
    fname = os.path.join(eval_dir, f'r{iteration}', 'tables', combined, f'iterevals.{fname_key}.csv')
    summary_table = pd.read_csv(fname, index_col = 0)
    summary_table = summary_table[[str(n) for n in range(1, iteration+1)]]
    
    print(summary_table)
    title = fname_key = '-'.join(sub_keys.values())
    fig = my_plot(summary_table, plot_args, title, ylim=ylim)
    
    if save_plot:
        assert not plot_type is None
        os.makedirs(os.path.join(eval_dir, f'r{iteration}', 'sum_plots', combined), exist_ok=True)
        fig.savefig(os.path.join(eval_dir, f'r{iteration}', 'sum_plots', combined, f'iterevals.{fname_key}.{plot_type}'))
        
        print(f"Saved at\n{os.path.join(eval_dir, f'r{iteration}', 'sum_plots')}")
    

# Summarize

In [5]:
model='roberta-large'
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
eval_dir = os.path.join(repo, 'eval_summary', model)

## Validation Sets

In [6]:
mod = 'separate'  # combined, hyp, separate, separate_hyp
iteration = 3
save_plot = False
plot_type = 'jpg'

ylim = [0.5,1.0]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_val_summary(mod, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3
baseline  0.843813  0.854545  0.862069
LotS      0.823529  0.787018  0.786720
LitL      0.850932  0.826804  0.807771


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Iterevals

In [7]:
glue_case_keys = pd.read_csv('glue_case_keys.csv', header=0, index_col=False)
hans_case_keys = pd.read_csv('hans_case_keys.csv', header=0, index_col=False)

In [8]:
'''
Cases for GLUE:
    combined, Knowledge, Lexical Semantics, Logic, Predicate-Argument Structure

Cases for HANS:
    combined, constituent, lexical_overlap, subsequence

'''

glue_case = 'Lexical Semantics'
print(glue_case_keys.loc[glue_case_keys['case'] == glue_case, 'subcase'])
# print('='*90 + '\n')

# hans_case = 'constituent'
# print(hans_case_keys.loc[hans_case_keys['case'] == hans_case, 'subcase'])

4                           combined
5                          Factivity
6              Factivity;Quantifiers
7                 Lexical entailment
8       Lexical entailment;Factivity
9     Lexical entailment;Quantifiers
10            Morphological negation
11                    Named entities
12                       Quantifiers
13                        Redundancy
14             Symmetry/Collectivity
Name: subcase, dtype: object


In [9]:
sub_keys = {
    'dataset': 'glue',     # either hans or glue
    'case': 'Logic',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'combined',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}
iteration = 2
save_plot = False
plot_type = 'jpg'

combined = 'combined' # either combined or separate
ylim = [0.6,0.75]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, combined, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2
baseline  0.541209  0.541209
LotS      0.519231  0.593407
LitL      0.546703  0.521978


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RuntimeError: libpng signaled error

# Save Plots

In [10]:
iteration = 5

## In-Distribution

In [11]:
mod = 'combined'  # combined, hyp, separate, separate_hyp
save_plot = True
plot_type = 'jpg'

ylim = [0.75,1.0]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_val_summary(mod, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.843813  0.854251  0.862255  0.869126  0.874496
LotS      0.823529  0.809331  0.811868  0.812057  0.808752
LitL      0.850932  0.833678  0.837337  0.836766  0.837938


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [12]:
mod = 'separate'  # combined, hyp, separate, separate_hyp

save_plot = True
plot_type = 'jpg'

ylim = [0.75,1.0]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_val_summary(mod, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.843813  0.854545  0.862069  0.861446  0.871743
LotS      0.823529  0.787018  0.786720  0.792261  0.803644
LitL      0.850932  0.826804  0.807771  0.837113  0.832298


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [13]:
mod = 'hyp'  # combined, hyp, separate, separate_hyp

save_plot = True
plot_type = 'jpg'

ylim = [0.5,0.75]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_val_summary(mod, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.602434  0.611336  0.634031  0.633148  0.636400
LotS      0.561866  0.563895  0.564396  0.561297  0.555916
LitL      0.561077  0.568182  0.583391  0.564367  0.563299


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [14]:
mod = 'separate_hyp'  # combined, hyp, separate, separate_hyp

save_plot = True
plot_type = 'jpg'

ylim = [0.5,0.75]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_val_summary(mod, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.602434  0.616162  0.653144  0.642570  0.655311
LotS      0.561866  0.549696  0.565392  0.541752  0.520243
LitL      0.561077  0.612371  0.562372  0.523711  0.552795


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


## Held-out

In [15]:
'''
Cases for GLUE:
    combined, Knowledge, Lexical Semantics, Logic, Predicate-Argument Structure

Cases for HANS:
    combined, constituent, lexical_overlap, subsequence

'''

glue_case = 'Lexical Semantics'
print(glue_case_keys.loc[glue_case_keys['case'] == glue_case, 'subcase'])
# print('='*90 + '\n')

# hans_case = 'constituent'
# print(hans_case_keys.loc[hans_case_keys['case'] == hans_case, 'subcase'])

4                           combined
5                          Factivity
6              Factivity;Quantifiers
7                 Lexical entailment
8       Lexical entailment;Factivity
9     Lexical entailment;Quantifiers
10            Morphological negation
11                    Named entities
12                       Quantifiers
13                        Redundancy
14             Symmetry/Collectivity
Name: subcase, dtype: object


## GLUE

In [16]:
sub_keys = {
    'dataset': 'glue',     # either hans or glue
    'case': 'Lexical Semantics',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'combined',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
ylim = [0.5,1.0]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.638587  0.690217  0.679348  0.682065  0.638587
LotS      0.622283  0.676630  0.668478  0.690217  0.665761
LitL      0.663043  0.679348  0.682065  0.679348  0.646739


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
                 1         2         3         4         5
baseline  0.638587  0.649457  0.627717  0.649457  0.641304
LotS      0.622283  0.630435  0.679348  0.646739  0.652174
LitL      0.663043  0.671196  0.608696  0.657609  0.649457


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [17]:
sub_keys = {
    'dataset': 'glue',     # either hans or glue
    'case': 'Knowledge',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'combined',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
ylim = [0.5,1.0]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.552817  0.602113  0.580986  0.598592  0.559859
LotS      0.598592  0.598592  0.595070  0.580986  0.605634
LitL      0.584507  0.566901  0.570423  0.573944  0.563380


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
                 1         2         3         4         5
baseline  0.552817  0.566901  0.549296  0.563380  0.580986
LotS      0.598592  0.602113  0.538732  0.580986  0.602113
LitL      0.584507  0.570423  0.552817  0.609155  0.559859


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [18]:
sub_keys = {
    'dataset': 'glue',     # either hans or glue
    'case': 'Logic',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'combined',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
ylim = [0.25,0.75]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.541209  0.541209  0.563187  0.587912  0.560440
LotS      0.519231  0.593407  0.582418  0.587912  0.582418
LitL      0.546703  0.521978  0.538462  0.563187  0.560440


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
                 1         2         3         4         5
baseline  0.541209  0.546703  0.513736  0.516484  0.532967
LotS      0.519231  0.554945  0.524725  0.587912  0.527473
LitL      0.546703  0.568681  0.486264  0.568681  0.513736


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [19]:
sub_keys = {
    'dataset': 'glue',     # either hans or glue
    'case': 'Predicate-Argument Structure',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'combined',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
ylim = [0.5,1.00]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1         2         3         4         5
baseline  0.617925  0.641509  0.653302  0.648585  0.658019
LotS      0.622642  0.691038  0.695755  0.700472  0.679245
LitL      0.599057  0.596698  0.617925  0.660377  0.610849


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
                 1         2         3         4         5
baseline  0.617925  0.582547  0.620283  0.634434  0.643868
LotS      0.622642  0.603774  0.691038  0.669811  0.634434
LitL      0.599057  0.613208  0.542453  0.648585  0.613208


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


## HANS

In [20]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'lexical_overlap',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'entailment',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

ylim = [0,1]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

ylim = [0,1.0]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

               1       2       3       4       5
baseline  0.7070  0.4708  0.5498  0.4476  0.4062
LotS      0.6056  0.4570  0.3322  0.3366  0.2734
LitL      0.5934  0.5270  0.5376  0.6062  0.5466


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
               1       2       3       4       5
baseline  0.7070  0.7634  0.5224  0.5400  0.4166
LotS      0.6056  0.6574  0.3826  0.3428  0.4884
LitL      0.5934  0.7368  0.8370  0.6722  0.6140


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [21]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'lexical_overlap',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'non-entailment',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

ylim = [0,1]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

ylim = [0,1.0]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

               1       2       3       4       5
baseline  0.3734  0.5394  0.5940  0.6364  0.6632
LotS      0.4874  0.7096  0.6546  0.7350  0.7906
LitL      0.4878  0.5992  0.5588  0.7422  0.6632


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
               1      2       3       4       5
baseline  0.3734  0.331  0.5080  0.4706  0.5536
LotS      0.4874  0.334  0.5990  0.6552  0.7500
LitL      0.4878  0.580  0.2638  0.7454  0.6166


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [22]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'subsequence',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'entailment',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

ylim = [0,1]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

ylim = [0,1.0]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

               1       2       3       4       5
baseline  0.4160  0.2626  0.2848  0.2406  0.1878
LotS      0.4884  0.2194  0.1914  0.1696  0.0872
LitL      0.5044  0.3414  0.3966  0.4182  0.5006


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
               1       2       3       4       5
baseline  0.4160  0.6068  0.2798  0.4010  0.3086
LotS      0.4884  0.6350  0.1526  0.2924  0.3904
LitL      0.5044  0.6700  0.8740  0.4032  0.5338


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [23]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'subsequence',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'non-entailment',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

ylim = [0,1]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

ylim = [0,1.0]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

               1       2       3       4       5
baseline  0.6936  0.9158  0.8978  0.8960  0.9152
LotS      0.8614  0.9144  0.9520  0.9462  0.9538
LitL      0.8508  0.9384  0.8930  0.8972  0.8652


C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
               1       2       3       4       5
baseline  0.6936  0.7676  0.8914  0.8348  0.9120
LotS      0.8614  0.6420  0.9578  0.9236  0.8932
LitL      0.8508  0.7798  0.4140  0.9224  0.7500


C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [24]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'constituent',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'entailment',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

ylim = [0,1]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

ylim = [0,1.0]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

               1       2       3       4       5
baseline  0.4912  0.3106  0.3574  0.3042  0.3084
LotS      0.2422  0.3414  0.2388  0.2486  0.2618
LitL      0.4238  0.2744  0.2050  0.3706  0.3562


C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
               1       2       3       4       5
baseline  0.4912  0.6032  0.4036  0.3718  0.4468
LotS      0.2422  0.6010  0.2778  0.3022  0.4186
LitL      0.4238  0.4188  0.7142  0.4116  0.6588


C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots


In [25]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'constituent',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'non-entailment',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}

save_plot = True
plot_type = 'jpg'

# combined = 'combined' # either combined or separate
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

ylim = [0,1]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'combined', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

ylim = [0,1.0]
display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, 'separate', ylim=ylim, save_plot=save_plot, plot_type=plot_type)

               1       2       3       4       5
baseline  0.3530  0.5162  0.5080  0.5630  0.6810
LotS      0.6204  0.4892  0.8564  0.7456  0.7096
LitL      0.5940  0.8124  0.7294  0.7678  0.7026


C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
               1       2       3       4       5
baseline  0.3530  0.3774  0.4204  0.5452  0.4686
LotS      0.6204  0.3102  0.6326  0.7086  0.5104
LitL      0.5940  0.6026  0.3384  0.5476  0.2706


C:\Users\Willi\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved at
C:\Users\Willi\Documents\NYU\2020_Fall\semantics_seminar\lip\ling_in_loop\eval_summary\r5\sum_plots
